In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# !pip install transformers sentencepiece accelerate -q

# !pip install optuna transformers -q
!pip install -r ../requirements.txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 9.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 12.1 MB/s eta 0:00:00


In [ ]:
import re
import emoji
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import DebertaV2TokenizerFast, DebertaV2Model, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
from ftfy import fix_text




2025-11-30 16:18:48.076244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764519528.261681      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764519528.319429      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# -------------------
# Seeds & Device
# -------------------
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

# -------------------
# Hyperparameters
# -------------------
MAX_LEN = 256
BATCH_SIZE = 8
EPOCHS = 15
LR = 1e-5
MODEL_NAME = "microsoft/deberta-v3-large"

In [ ]:
data_path= '../Data/train.csv'

In [ ]:
df = pd.read_csv(data_path= '../Data/train.csv')
X = df['text'].tolist()
y = df[['anger','fear','joy','sadness','surprise']].values

def clean_text(text):
    text = fix_text(text)
    text = re.sub(r'http\S+|www.\S+', '<URL>', text)
    text = re.sub(r'@\w+', '<USER>', text)
    text = emoji.demojize(text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

X = [clean_text(x) for x in X]

train_texts, val_texts, train_labels, val_labels = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)


In [6]:
tokenizer = DebertaV2TokenizerFast.from_pretrained(MODEL_NAME)

def encode_batch(texts):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )

train_enc = encode_batch(train_texts)
val_enc   = encode_batch(val_texts)


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

In [7]:
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float32)
        return item

train_dataset = EmotionDataset(train_enc, train_labels)
val_dataset   = EmotionDataset(val_enc, val_labels)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE)


In [8]:
class EmotionClassifier(nn.Module):
    def __init__(self, num_labels=5):
        super().__init__()
        self.deberta = DebertaV2Model.from_pretrained(MODEL_NAME)
        hidden_size = self.deberta.config.hidden_size

        self.dropout = nn.Dropout(0.3)
        self.norm = nn.LayerNorm(hidden_size)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        out = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden = out.last_hidden_state

        # Mean pooling
        masked = hidden * attention_mask.unsqueeze(-1)
        pooled = masked.sum(1) / attention_mask.sum(1, keepdim=True)

        x = self.norm(pooled)
        x = self.dropout(x)
        return self.classifier(x)

model = EmotionClassifier(num_labels=5).to(device)


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

In [9]:
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.AdamW([
    {"params": model.deberta.parameters(), "lr": LR},
    {"params": model.classifier.parameters(), "lr": 3e-5}
])

num_training_steps = len(train_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps
)

scaler = torch.cuda.amp.GradScaler()


In [10]:
def train_epoch(model, loader, optimizer, scheduler, criterion):
    model.train()
    total_loss = 0
    for batch in tqdm(loader, leave=False, desc="Training"):
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        y = batch['labels'].to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            logits = model(ids, mask)
            loss = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        total_loss += loss.item()

        # Correct wandb logging
        # wandb.log({
        #     "lr": optimizer.param_groups[0]["lr"],
        #     "train_loss": loss.item(),
        #     "total_loss": total_loss / (len(loader))  # optional: average so far
        # })

    return total_loss / len(loader)


In [11]:
def evaluate(model, loader, criterion, threshold=0.5):
    model.eval()
    preds, trues = [], []
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(loader, leave=False, desc="Validating"):
            ids = batch['input_ids'].to(device)
            mask = batch['attention_mask'].to(device)
            y = batch['labels'].to(device)

            logits = model(ids, mask)
            loss = criterion(logits, y)
            total_loss += loss.item()

            probs = torch.sigmoid(logits)
            pred = (probs > threshold).long()
            preds.extend(pred.cpu().tolist())
            trues.extend(y.cpu().tolist())

    val_loss = total_loss / len(loader)
    val_f1 = f1_score(trues, preds, average='macro')

    # WandB logging
    # wandb.log({
    #     "val_loss": val_loss,
    #     "val_f1": val_f1
    # })

    return val_loss, val_f1


In [ ]:
best_f1 = 0.0

for epoch in range(EPOCHS):
    train_loss = train_epoch(model, train_loader, optimizer, scheduler, criterion)
    val_loss, val_f1 = evaluate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")

    # Saving the best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), "best_model_deberta.pt")
        print(f"Saved best model. Val_f1: {val_f1} Loss: {train_loss}")


### Uploading the model to KaggleHub:

In [ ]:
# from kaggle_secrets import UserSecretsClient

# # # Load kaggle.json secret
# user_secrets = UserSecretsClient()
# kaggle_json = user_secrets.get_secret("kaggle_json")

# # # print("Loaded Key:", kaggle_json)

In [ ]:
# # Parse and set environment variables
# kaggle_data = json.loads(kaggle_json)
# os.environ["KAGGLE_USERNAME"] = kaggle_data["username"]
# os.environ["KAGGLE_KEY"] = kaggle_data["key"]

# print("✅ Kaggle credentials set successfully!")

In [ ]:
# torch.save(current_model.state_dict(),f"best_model_{name}.pth")
# print("Best Model Saved!")

In [ ]:
import kagglehub
import os, json
from pathlib import Path

# Path to your saved model
model_path = Path(f"/kaggle/working/best_model_deberta.pt")

# Define handle -> format: "username/model_name/framework/version"
handle = f"{os.environ['KAGGLE_USERNAME']}/dl-GenAI-Project-DeBERTa/pytorch/v1"

# Upload model
kagglehub.model_upload(handle, str(model_path), version_notes="First version upload 🚀")

print(f"✅ DeBERTa Model successfully uploaded to KaggleHub!")

### On to the test data

In [ ]:
df_test = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/test.csv")
df_test.dropna(subset=['text'], inplace=True)
df_test = df_test[['text']]
df_test['text'] = df_test['text'].apply(clean_text)

In [ ]:
# Importing required libraries
# # ----------------------------
import kagglehub
import os, json
from pathlib import Path
import torch
import torch.nn as nn
from transformers import DebertaV2Model


In [ ]:
# # ----------------------------
# # 2️⃣ Downloading model from KaggleHub
# # ----------------------------
# handle = f"{os.environ['KAGGLE_USERNAME']}/dl-GenAI-Project-DeBERTa/pytorch/v1"
# # handle = "anshuraj22f3000757/dl-GenAI-Project-DeBERTa/pytorch/v1"
handle = "anshuraj22f3000757/dl-GenAI-Project-DeBERTa/pytorch/v2"
# handle = "anshuraj22f3000757/dl-GenAI-Project-RoBERTa/pytorch/v1"

download_path = kagglehub.model_download(handle)
print("Model downloaded to:", download_path)


In [ ]:

# ----------------------------
# 3️⃣ Defining the same EmotionClassifier class
# ----------------------------
MODEL_NAME = "microsoft/deberta-v3-large"

class EmotionClassifier(nn.Module):
    def __init__(self, num_labels=5):
        super().__init__()
        self.deberta = DebertaV2Model.from_pretrained(MODEL_NAME)
        hidden_size = self.deberta.config.hidden_size

        self.dropout = nn.Dropout(0.3)
        self.norm = nn.LayerNorm(hidden_size)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        out = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden = out.last_hidden_state

        # Mean pooling
        masked = hidden * attention_mask.unsqueeze(-1)
        pooled = masked.sum(1) / attention_mask.sum(1, keepdim=True)

        x = self.norm(pooled)
        x = self.dropout(x)
        return self.classifier(x)

In [ ]:

#  Initializing the model & setting to device

device = "cuda" if torch.cuda.is_available() else "cpu"
model = EmotionClassifier(num_labels=5).to(device)


In [ ]:

# Find the .pt file inside the downloaded folder

pt_files = list(Path(download_path).rglob("*.pt"))
assert len(pt_files) > 0, "No .pt file found!"
model_file = str(pt_files[0])
print("Found model file:", model_file)

In [ ]:
# Loading weights into model
model.load_state_dict(torch.load(model_file, map_location=device))
print("✅ DeBERTa model loaded successfully!")

In [ ]:
test_enc = tokenizer(
    df_test['text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=MAX_LEN,
    return_tensors=None
)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {k: torch.tensor(v[idx], dtype=torch.long) for k, v in self.encodings.items()}

test_dataset = TestDataset(test_enc)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model.eval()
all_preds = []
best_threshold = 0.42  

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)

        logits = model(ids, mask)
        probs = torch.sigmoid(logits)
        pred = (probs > best_threshold).int()
        all_preds.append(pred.cpu())

predictions = torch.cat(all_preds, dim=0)
print("Predictions Done!!!")
print("Predictions shape:", predictions.shape)

In [ ]:
# Save CSV
emotion_cols = ['anger','fear','joy','sadness','surprise']
df_preds = pd.DataFrame(predictions.numpy(), columns=emotion_cols)
df_preds.insert(0, 'id', np.arange(len(df_preds)))
df_preds.to_csv("submission.csv", index=False)

In [ ]:
torch.cuda.empty_cache()

# Cleaining the GPU's history which ensures that it doesn't run slow with time.